In [1]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

## Preprocessing

In [2]:
def resize(input_image, input_mask):
   input_image = tf.image.resize(input_image, (128, 128), method="nearest")
   input_mask = tf.image.resize(input_mask, (128, 128), method="nearest")
   return input_image, input_mask

def augment(input_image, input_mask):
   if tf.random.uniform(()) > 0.5:
       # Random flipping of the image and mask
       input_image = tf.image.flip_left_right(input_image)
       input_mask = tf.image.flip_left_right(input_mask)
   return input_image, input_mask


def normalize(input_image, input_mask):
   input_image = tf.cast(input_image, tf.float32) / 255.0
   input_mask -= 1
   return input_image, input_mask

In [ ]:
!wget https://mm.cs.uec.ac.jp/uecfoodpix/UECFOODPIXCOMPLETE.tar

In [ ]:
!wget https://storage.googleapis.com/food201/food201.zip

In [ ]:
!unzip food201.zip

In [56]:
from sklearn.model_selection import train_test_split
import os
import shutil

src_dir = './images'

output_dir = './seg-dataset'

food201_masks_dir='./food201/pixel_annotations'

test_size = 0.1

shutil.rmtree(output_dir, ignore_errors=True)


for class_name in os.listdir(src_dir):
    class_dir = os.path.join(src_dir, class_name)

    image_files = [os.path.join(class_dir, f) for f in os.listdir(class_dir) if os.path.isfile(os.path.join(class_dir, f))]

    train_files, test_files = train_test_split(image_files, test_size=test_size)
    
    # print(train_files)
    
    
    # Create 4 directories 
    train_image_dir = os.path.join(output_dir, 'train-image', class_name)
    test_image_dir = os.path.join(output_dir, 'test-image', class_name)
    train_mask_dir=os.path.join(output_dir, 'train-mask', class_name)
    test_mask_dir=os.path.join(output_dir, 'test-mask', class_name)
    
    os.makedirs(train_image_dir, exist_ok=True)
    os.makedirs(test_image_dir, exist_ok=True)
    os.makedirs(train_mask_dir, exist_ok=True)
    os.makedirs(test_mask_dir, exist_ok=True)
    
    
    # for f in train_files:
    #     shutil.copy(f, os.path.join(train_class_dir, os.path.basename(f)))

#     for f in test_files:
#         shutil.copy(f, os.path.join(test_class_dir, os.path.basename(f)))
        
    
    for f in train_files:
        file_name = os.path.basename(f)[:-4]  # Remove the file extension
        mask_file = os.path.join(food201_masks_dir, class_name,file_name + '.png')

        # Copy the image file to the train directory
        shutil.copy(f, os.path.join(train_image_dir, os.path.basename(f)))

        # Copy the mask file to the train mask directory
        shutil.copy(mask_file, os.path.join(train_mask_dir, os.path.splitext(os.path.basename(f))[0] + '.png'))

    for f in test_files:
        file_name = os.path.basename(f)[:-4]  # Remove the file extension
        mask_file = os.path.join(food201_masks_dir, class_name,file_name + '.png')

        # Copy the image file to the train directory
        shutil.copy(f, os.path.join(test_image_dir, os.path.basename(f)))

        # Copy the mask file to the train mask directory
        shutil.copy(mask_file, os.path.join(test_mask_dir, os.path.splitext(os.path.basename(f))[0] + '.png'))

In [17]:
# MERGE UECFOODPIXCOMPLETE with Food201

import os
import shutil

output_dir = './seg-dataset'

uec_train_image_dir='./UECFOODPIXCOMPLETE/data/UECFoodPIXCOMPLETE/train/img'
uec_train_mask_dir='./UECFOODPIXCOMPLETE/data/UECFoodPIXCOMPLETE/train/mask'

uec_test_image_dir='./UECFOODPIXCOMPLETE/data/UECFoodPIXCOMPLETE/test/img'
uec_test_mask_dir='./UECFOODPIXCOMPLETE/data/UECFoodPIXCOMPLETE/test/mask'

uec_test_dir='./UECFoodPIXCOMPLETE/test/img'
class_name="uec"

train_image_dir = os.path.join(output_dir, 'train-image', class_name)
test_image_dir = os.path.join(output_dir, 'test-image', class_name)
train_mask_dir=os.path.join(output_dir, 'train-mask', class_name)
test_mask_dir=os.path.join(output_dir, 'test-mask', class_name)

os.makedirs(train_image_dir, exist_ok=True)
os.makedirs(test_image_dir, exist_ok=True)
os.makedirs(train_mask_dir, exist_ok=True)
os.makedirs(test_mask_dir, exist_ok=True)


for f in os.listdir(uec_test_image_dir):
        file_name = os.path.basename(f)[:-4]  # Remove the file extension
        image_file=mask_file = os.path.join(uec_test_image_dir,file_name + '.jpg')
        mask_file = os.path.join(uec_test_mask_dir,file_name + '.png')

        # Copy the image file to the train directory
        shutil.copy(image_file, os.path.join(test_image_dir, os.path.basename(f)))
        

        # Copy the mask file to the train mask directory
        shutil.copy(mask_file, os.path.join(test_mask_dir, os.path.splitext(os.path.basename(f))[0] + '.png'))

In [61]:
# Check all images exist after splitting

train_image_dir = './seg-dataset/train-image'
train_mask_dir = './seg-dataset/train-mask'

test_image_dir = './seg-dataset/test-image'
test_mask_dir = './seg-dataset/test-mask'

food201_masks_dir='./food201/pixel_annotations'

def get_image_filenames(image_dir):
        image_filenames = []
        for class_dir in os.listdir(image_dir):
            class_image_dir = os.path.join(image_dir, class_dir)
            class_image_filenames = os.listdir(class_image_dir)
            for image_filename in class_image_filenames:
                image_filenames.append(os.path.join(class_dir, image_filename))
        return len(image_filenames)
    
    
train_image=get_image_filenames(train_image_dir)
train_mask=get_image_filenames(train_mask_dir)
test_image=get_image_filenames(test_image_dir)
test_mask=get_image_filenames(test_mask_dir)

food201_masks=get_image_filenames(food201_masks_dir)

# Check every train image has an equuiavent train mask and same for test images

def check_missing_images(image_dir, mask_dir):
        image_filenames = []
        for class_dir in os.listdir(image_dir):
            class_image_dir = os.path.join(image_dir, class_dir)
            class_image_filenames = os.listdir(class_image_dir)
            for image_filename in class_image_filenames:
                image_path = os.path.join(class_image_dir, image_filename)
                mask_filename = image_filename.split('.')[0] + '.png'
                mask_path = os.path.join(mask_dir, class_dir, mask_filename)
                if os.path.isfile(mask_path):
                    image_filenames.append(os.path.join(class_dir, image_filename))
                else:
                    print(f"Warning: No mask found for image '{image_filename}'")
        return len(image_filenames)

print(check_missing_images(train_image_dir, train_mask_dir))
print(check_missing_images(test_image_dir, test_mask_dir))


10839
10839
1254
1254
12093
----------
10839
1254


In [2]:
import os
import cv2
import numpy as np
from tensorflow.keras.utils import Sequence
import csv
import random



class DataGenerator(Sequence):
    def __init__(self, image_dir, mask_dir, batch_size, shuffle=True):
        self.image_dir = image_dir
        self.mask_dir = mask_dir
        self.batch_size = batch_size
        self.classes = os.listdir(image_dir)
        self.num_classes = len(self.classes)
        self.image_filenames = self.get_image_filenames()
        self.num_samples = len(self.image_filenames)
        self.image_size=(224,224)
        self.shuffle = shuffle
        self.indexes = np.arange(len(self.image_filenames))
        self.food_item_ids = self.get_food_item_ids()
        # self.transform=self.get_transform()
        self.on_epoch_end()
        
    
    def get_food_item_ids(self):
        csv_file = './food201/pixel_annotations_map.csv'  
        food_ids = []

        with open(csv_file, 'r') as file:
            reader = csv.reader(file)
            for row in reader:
                if row:  # Skip empty rows
                    id = int(row[0].strip())
                    food_ids.append(id)
        return food_ids
        
        
    def get_image_filenames(self):
        image_filenames = []
        for class_dir in self.classes:
            # if class_dir !="uec":
            #     continue
            class_image_dir = os.path.join(self.image_dir, class_dir)
            class_image_filenames = os.listdir(class_image_dir)
            for image_filename in class_image_filenames:
                image_filenames.append(os.path.join(class_dir, image_filename))
            # print(image_filenames)
        return image_filenames
    
    def __len__(self):
        return int(np.ceil(self.num_samples / float(self.batch_size)))
    
    def on_epoch_end(self):
        """Callback function to shuffle indexes each epoch"""
        if self.shuffle:
            self.indexes = np.random.permutation(self.indexes)
    
    def __getitem__(self, idx):
        if not self.shuffle:
            batch_image_filenames = self.image_filenames[idx * self.batch_size:(idx + 1) * self.batch_size]
        else:
            batch_indexes = self.indexes[idx * self.batch_size:(idx + 1) * self.batch_size]
            batch_image_filenames = [self.image_filenames[i] for i in batch_indexes]
        batch_images = []
        batch_masks = []
        
        for image_filename in batch_image_filenames:
            # Paths
            image_path = os.path.join(self.image_dir, image_filename)
            
            mask_filename = image_filename.split('.')[0] + '.png'
            mask_path = os.path.join(self.mask_dir, mask_filename)
            
            is_uec = image_filename.split("/")[0] == "uec"

            # Preprocess Image
            image = cv2.imread(image_path)
            image = cv2.resize(image, self.image_size)
            
            # Preprocess Mask according to which dataset it belongs to
            mask=None
            
            if is_uec:
                mask = cv2.imread(mask_path)
                mask = cv2.cvtColor(mask, cv2.COLOR_BGR2GRAY)
                mask = cv2.resize(mask, self.image_size)
                mask = np.where(mask != 0, 1, 0)
                
            else: # Food201 dataset
                mask = cv2.imread(mask_path, cv2.IMREAD_GRAYSCALE)
                mask = cv2.resize(mask, self.image_size)
                mask = np.where(np.isin(mask, self.food_item_ids), 1, 0) # Threshold the mask to convert it to binar

             # Normalize the image
            image = image / 255.0
            
            batch_images.append(image)
            batch_masks.append(mask)
        
        return np.array(batch_images), np.array(batch_masks)
    

train_image_dir = './seg-dataset/train-image'
train_mask_dir = './seg-dataset/train-mask'

test_image_dir = './seg-dataset/test-image'
test_mask_dir = './seg-dataset/test-mask'

batch_size=32
img_height=224
img_width=224

train_generator = DataGenerator(train_image_dir, train_mask_dir, batch_size)
val_generator = DataGenerator(test_image_dir, test_mask_dir, batch_size, shuffle=False)

num_training_samples=train_generator.num_samples
num_validation_samples = val_generator.num_samples

print(num_training_samples)
print(num_validation_samples)


19839
2254


In [2]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Set the directories for the training data and the masks
train_orig_dir = './seg-dataset/train-original'
train_mask_dir = './seg-dataset/train-mask'

batch_size=16
img_height=224
img_width=224

def threshold_mask(mask):
    mask[mask > 0] = 1
    mask[mask <= 0] = 0
    return mask

# Create an ImageDataGenerator for the original images
data_gen_args = dict(rescale=1./255)
image_datagen = ImageDataGenerator(**data_gen_args)
image_generator = image_datagen.flow_from_directory(train_orig_dir,
                                                     class_mode=None,
                                                     batch_size=batch_size,
                                                     target_size=(img_height, img_width))

mask_datagen = ImageDataGenerator(**data_gen_args, preprocessing_function=threshold_mask)
mask_generator = mask_datagen.flow_from_directory(train_mask_dir,
                                                   class_mode=None,
                                                   batch_size=batch_size,
                                                   target_size=(img_height, img_width),
                                                   color_mode='grayscale')


num_training_samples=len(mask_generator)

# Combine the generators into one that yields image and mask pairs
train_generator = zip(image_generator, mask_generator)

# num_training_samples=len(train_generator)

# print(f"Num of training samples {num_training_samples}")


# Set the directories for the test data and the masks
test_orig_dir = './seg-dataset/test-original'
test_mask_dir = './seg-dataset/test-mask'

# Create an ImageDataGenerator for the original test images
test_image_datagen = ImageDataGenerator(**data_gen_args)
test_image_generator = test_image_datagen.flow_from_directory(test_orig_dir,
                                                               class_mode=None,
                                                               batch_size=batch_size,
                                                               target_size=(img_height, img_width))

# Create an ImageDataGenerator for the test masks
test_mask_datagen = ImageDataGenerator(**data_gen_args, preprocessing_function=threshold_mask)
test_mask_generator = test_mask_datagen.flow_from_directory(test_mask_dir,
                                                             class_mode=None,
                                                             batch_size=batch_size,
                                                             target_size=(img_height, img_width),
                                                             color_mode='grayscale')

num_validation_samples=len(test_mask_generator)

# Combine the generators into one that yields test image and mask pairs
test_generator = zip(test_image_generator, test_mask_generator)




Found 10839 images belonging to 101 classes.
Found 10839 images belonging to 101 classes.
Found 1254 images belonging to 101 classes.
Found 1254 images belonging to 101 classes.


## Model 1

In [2]:
from tensorflow.keras import layers

def double_conv_block(x, n_filters):
   # Conv2D then ReLU activation
   x = layers.Conv2D(n_filters, 3, padding = "same", kernel_initializer = "he_normal")(x)
   x = layers.BatchNormalization()(x)
   # x = layers.Activation("relu")(x)
   x = layers.LeakyReLU(alpha=0.2)(x)


   # Conv2D then ReLU activation
   x = layers.Conv2D(n_filters, 3, padding = "same", kernel_initializer = "he_normal")(x)
   x = layers.BatchNormalization()(x)
   x = layers.LeakyReLU(alpha=0.2)(x)
   x = layers.Dropout(0.2)(x)
   # x = layers.Activation("relu")(x)

   return x


def downsample_block(x, n_filters):
   f = double_conv_block(x, n_filters)
   p = layers.MaxPool2D(2)(f)
   # p = layers.Dropout(0.3)(p)
   return f, p


def upsample_block(x, conv_features, n_filters):
   # upsample
   x = layers.Conv2DTranspose(n_filters, 3, 2, padding="same")(x)
   # concatenate
   x = layers.concatenate([x, conv_features])
   # dropout
   # x = layers.Dropout(0.3)(x)
   # Conv2D twice with ReLU activation
   x = double_conv_block(x, n_filters)
   return x

In [5]:
def build_unet_model():
     # inputs
   inputs = layers.Input(shape=(224,224,3))

   # encoder: contracting path - downsample
   # 1 - downsample
   f1, p1 = downsample_block(inputs, 64)
   # 2 - downsample
   f2, p2 = downsample_block(p1, 128)
   # 3 - downsample
   f3, p3 = downsample_block(p2, 256)
   # 4 - downsample
   f4, p4 = downsample_block(p3, 512)

   # 5 - bottleneck
   bottleneck = double_conv_block(p4, 1024)

   # decoder: expanding path - upsample
   # 6 - upsample
   u6 = upsample_block(bottleneck, f4, 512)
   # 7 - upsample
   u7 = upsample_block(u6, f3, 256)
   # 8 - upsample
   u8 = upsample_block(u7, f2, 128)
   # 9 - upsample
   u9 = upsample_block(u8, f1, 64)

   # outputs
   outputs = layers.Conv2D(1, 1, padding="same", activation = "sigmoid")(u9)

   # unet model with Keras Functional API
   unet_model = tf.keras.Model(inputs, outputs, name="U-Net")

   return unet_model

## Model 2

## Training

In [6]:
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.callbacks import EarlyStopping, CSVLogger, ModelCheckpoint, ReduceLROnPlateau
from tensorflow.keras.optimizers import Adam, SGD
from tensorflow.keras.utils import plot_model


model=build_unet_model()

# model=build_unet_model_pretrained()
# tf.keras.utils.plot_model(model, "model.png", show_shapes=False, show_dtype=False, show_layer_names=True, rankdir='TB', expand_nested=False, dpi=96)

model.compile(optimizer=Adam(1e-3),
                  loss="binary_crossentropy",
                  metrics=[ tf.keras.metrics.BinaryAccuracy() , tf.keras.metrics.BinaryIoU()]
              # tf.keras.metrics.Recall(name="recall"),
              # tf.keras.metrics.Precision(name="precision"),
              )


csv_logger=CSVLogger('segmentation-unet.log')

checkpoint_callback = ModelCheckpoint(filepath='./U-Net/model_weights.{epoch:02d}.h5', 
                                       save_weights_only=True)


early_stopping = EarlyStopping(
    monitor='val_loss',    
    patience=10,            
    restore_best_weights=True   
)

epochs=100

reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2,
                              patience=5)

model_history = model.fit(train_generator,
                    validation_steps=num_validation_samples // batch_size,
                    validation_data=val_generator,
                    steps_per_epoch = num_training_samples // batch_size,
                    epochs=epochs,
                    verbose=2,
                    callbacks=[early_stopping, csv_logger, checkpoint_callback, reduce_lr])

## Resume Training

In [6]:
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.callbacks import EarlyStopping, CSVLogger, ModelCheckpoint
from tensorflow.keras.optimizers import Adam, SGD
from tensorflow.keras.utils import plot_model



model=build_unet_model()
model.load_weights('/notebooks/U-Net/model_weights.20.h5')


model.compile(optimizer=Adam(1e-3),
                  loss="binary_crossentropy",
                  metrics=[ tf.keras.metrics.BinaryAccuracy() , tf.keras.metrics.BinaryIoU()]
              )


csv_logger=CSVLogger('segmentation-unet-continue.log')


checkpoint_callback = ModelCheckpoint(filepath='./U-Net/model_weights.{epoch:02d}.h5', 
                                       save_weights_only=True)


early_stopping = EarlyStopping(
    monitor='val_loss',    
    patience=10,            
    restore_best_weights=True   
)

epochs=100

model_history = model.fit(train_generator,
                    validation_steps=num_validation_samples // batch_size,
                    validation_data=val_generator,
                    steps_per_epoch = num_training_samples // batch_size,
                    epochs=epochs,
                    initial_epoch=20,
                    verbose=2,
                    callbacks=[early_stopping, csv_logger, checkpoint_callback])

Epoch 21/100


2023-05-19 11:21:24.607141: W tensorflow/core/common_runtime/bfc_allocator.cc:290] Allocator (GPU_0_bfc) ran out of memory trying to allocate 3.04GiB with freed_by_count=0. The caller indicates that this is not a failure, but this may mean that there could be performance gains if more memory were available.
2023-05-19 11:21:24.607231: W tensorflow/core/common_runtime/bfc_allocator.cc:290] Allocator (GPU_0_bfc) ran out of memory trying to allocate 3.04GiB with freed_by_count=0. The caller indicates that this is not a failure, but this may mean that there could be performance gains if more memory were available.
2023-05-19 11:21:25.100905: W tensorflow/core/common_runtime/bfc_allocator.cc:290] Allocator (GPU_0_bfc) ran out of memory trying to allocate 2.60GiB with freed_by_count=0. The caller indicates that this is not a failure, but this may mean that there could be performance gains if more memory were available.
2023-05-19 11:21:25.100979: W tensorflow/core/common_runtime/bfc_allocato

619/619 - 435s - loss: 0.1825 - binary_accuracy: 0.9286 - binary_io_u: 0.8652 - val_loss: 0.2591 - val_binary_accuracy: 0.9067 - val_binary_io_u: 0.8280 - 435s/epoch - 703ms/step
Epoch 22/100
619/619 - 401s - loss: 0.1751 - binary_accuracy: 0.9311 - binary_io_u: 0.8695 - val_loss: 0.2543 - val_binary_accuracy: 0.9033 - val_binary_io_u: 0.8213 - 401s/epoch - 648ms/step
Epoch 23/100
619/619 - 407s - loss: 0.1640 - binary_accuracy: 0.9360 - binary_io_u: 0.8782 - val_loss: 0.2570 - val_binary_accuracy: 0.9064 - val_binary_io_u: 0.8273 - 407s/epoch - 657ms/step
Epoch 24/100
619/619 - 401s - loss: 0.1559 - binary_accuracy: 0.9392 - binary_io_u: 0.8838 - val_loss: 0.2749 - val_binary_accuracy: 0.9060 - val_binary_io_u: 0.8264 - 401s/epoch - 648ms/step
Epoch 25/100
619/619 - 401s - loss: 0.1459 - binary_accuracy: 0.9438 - binary_io_u: 0.8920 - val_loss: 0.2856 - val_binary_accuracy: 0.8996 - val_binary_io_u: 0.8149 - 401s/epoch - 648ms/step
Epoch 26/100
619/619 - 401s - loss: 0.1371 - binary_a

## Prediction

In [ ]:

# model.save('resnet50V2_model.h5')
# model = load_model('resnet50V2_model.h5')


model.load_weights('./model_weights.20.h5')

import matplotlib.pyplot as plt
import numpy as np
from tensorflow.keras.preprocessing import image

img_height=224
img_width=224

# Load the input image
img_path = '../test_images/sin3.jpeg'
img = image.load_img(img_path, target_size=(img_height, img_width))

# Convert the image to a numpy array and normalize its pixel values
x = image.img_to_array(img)
x /= 255.

# Add a batch dimension to the input
x = np.expand_dims(x, axis=0)

# Use the trained model to predict the mask
predicted_mask = model.predict(x)[0]

threshold = 0.5  # Adjust this threshold as needed
predicted_mask = (predicted_mask >= threshold).astype(np.uint8)

# # Plot the input image and the predicted mask side-by-side
fig, ax = plt.subplots(1, 2, figsize=(10, 5))
ax[0].imshow(img)
ax[1].imshow(predicted_mask, cmap='gray')


plt.show()